<a href="https://colab.research.google.com/github/BogereMark879/Bogere-mark/blob/main/Tourism_Experience_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the required libraries

import pandas as pd
import numpy as np

import nltk
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# load the data sets
df_attractions = pd.read_csv("/content/drive/MyDrive/tourism_attractions.csv",usecols=['item_id', 'name', 'experience_tags'])
df_preferences = pd.read_csv('/content/drive/MyDrive/userA_preferences.csv')

In [ ]:
# display the attractions dataframe
df_attractions

,item_id,name,experience_tags
0,1,Bwindi Forest National Park,"Gorilla trekking, terrestrial animals, water s..."
1,2,Queen Elizabeth National Park,"Terrestrial animals, aquatic animals, beautifu..."
2,3,Murchison Falls National Park,"Terrestrial animals, water sports"
3,4,Source of the Nile River,"Water sports, beautiful scenery, cultural inst..."
4,5,Kibale Forest National Park,"Terrestrial animals, bird watching"
5,6,Kidepo Valley National Park,"Terrestrial animals, mountain sports, beautifu..."
6,7,Rwenzori Mountains National Park,"Terrestrial animals, mountain sports, beautifu..."
7,8,Lake Victoria,"Beautiful scenery, water sports, bird watching..."
8,9,Lake Bunyonyi,"Water sports, beautiful scenery, indigenous cu..."
9,10,Kampala Capital City,"Beautiful scenery, indigenous culture, cultura..."


In [ ]:
# display user A's tourism experience preferences
df_preferences

,user_id,preferences
0,1,"Terrestrial animals, aquatic animals, mountain..."


In [ ]:
!cat configs/sample_input_config.pbtxt

cat: configs/sample_input_config.pbtxt: No such file or directory


In [ ]:
# function to process the text data
import nltk
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# run the function on the appropriate features to pre-process the two data sets and convert them into lists
import nltk
nltk.download('punkt')
norm_corpus_attractions = normalize_corpus(list(df_attractions['experience_tags']))
norm_corpus_preferences = normalize_corpus(list(df_preferences['preferences']))

#len(norm_corpus_attractions)
#len(norm_corpus_preferences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1

In [ ]:
# use TfidfVectorizer to vectorise

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)

tfidf_matrix_attractions = tfidf_vectorizer.fit_transform(norm_corpus_attractions)
tfidf_matrix_preferences = tfidf_vectorizer.transform(norm_corpus_preferences)

#tfidf_attractions.shape
#tfidf_preferences.shape

In [ ]:
# compute the cosine similarity

#print(cosine_similarity(tfidf_matrix_attractions, tfidf_matrix_preferences))
cs = cosine_similarity(tfidf_matrix_attractions, tfidf_matrix_preferences)
#convert to Pandas dataframe
df_cs = pd.DataFrame(cs)
df_cs.index = df_cs.index +1
df_cs.index.name = 'item_id'
df_cs = df_cs.rename(columns={0: 'similarity_score'})
df_cs

,similarity_score
item_id,
1,0.150245
2,0.614907
3,0.318366
4,0.044601
5,0.270621
6,0.574915
7,0.608965
8,0.040632
9,0.047855


In [ ]:
# merge the two files
df_attractions_with_similarity_scores = pd.merge(df_attractions, df_cs, on='item_id')
df_attractions_with_similarity_scores

,item_id,name,experience_tags,similarity_score
0,1,Bwindi Forest National Park,"Gorilla trekking, terrestrial animals, water s...",0.150245
1,2,Queen Elizabeth National Park,"Terrestrial animals, aquatic animals, beautifu...",0.614907
2,3,Murchison Falls National Park,"Terrestrial animals, water sports",0.318366
3,4,Source of the Nile River,"Water sports, beautiful scenery, cultural inst...",0.044601
4,5,Kibale Forest National Park,"Terrestrial animals, bird watching",0.270621
5,6,Kidepo Valley National Park,"Terrestrial animals, mountain sports, beautifu...",0.574915
6,7,Rwenzori Mountains National Park,"Terrestrial animals, mountain sports, beautifu...",0.608965
7,8,Lake Victoria,"Beautiful scenery, water sports, bird watching...",0.040632
8,9,Lake Bunyonyi,"Water sports, beautiful scenery, indigenous cu...",0.047855
9,10,Kampala Capital City,"Beautiful scenery, indigenous culture, cultura...",0.000000


In [ ]:
# sort the results in descending order
df_attractions_with_similarity_scores = df_attractions_with_similarity_scores.sort_values(by='similarity_score', ascending=False)

# extract list of recommendations
recommendations_list = df_attractions_with_similarity_scores['name'].tolist()
preference_list = ['Terrestrial animals', 'Aquatic animals', 'Water sports']

# print the top 5 recommended places
#print('Top 5 recommended places to visit in Uganda:', recommendations_list[0:5])

In [ ]:
df_attractions_with_similarity_scores

,item_id,name,experience_tags,similarity_score
1,2,Queen Elizabeth National Park,"Terrestrial animals, aquatic animals, beautifu...",0.614907
11,12,Lake Mburo National Park,"Terrestrial animals, aquatic animals, beautifu...",0.614907
6,7,Rwenzori Mountains National Park,"Terrestrial animals, mountain sports, beautifu...",0.608965
5,6,Kidepo Valley National Park,"Terrestrial animals, mountain sports, beautifu...",0.574915
14,15,Ziwa Rhino Sanctuary,"Terrestrial animals, aquatic animals, beautifu...",0.519217
2,3,Murchison Falls National Park,"Terrestrial animals, water sports",0.318366
13,14,Mount Elgon National Park,"Mountain sports, beautiful scenery, bird watching",0.313806
4,5,Kibale Forest National Park,"Terrestrial animals, bird watching",0.270621
12,13,Semuliki National Park,"Terrestrial animals, indigenous culture, bird ...",0.204572
10,11,Mgahinga Gorilla National Park,"Gorilla trekking, mountain sports, beautiful s...",0.163728


In [ ]:
# print and format the recommendation list
N = 5
top_5_recommendations = []
for index in range(0, N):
    top_5_recommendations.append(recommendations_list[index])

print(' ')
print('Based on your selection of preferred tourism experience;')
print(' ')
print(*('{}. {}'.format(*k) for k in enumerate(preference_list, start = 1)), sep="\n")
print(' ')
print('Top 5 recommended places to visit in Uganda are:')
print(' ')
print(*('{}. {}'.format(*k) for k in enumerate(top_5_recommendations, start = 1)), sep="\n")

 
Based on your selection of preferred tourism experience;
 
1. Terrestrial animals
2. Aquatic animals
3. Water sports
 
Top 5 recommended places to visit in Uganda are:
 
1. Queen Elizabeth National Park
2. Lake Mburo National Park
3. Rwenzori Mountains National Park
4. Kidepo Valley National Park
5. Ziwa Rhino Sanctuary


In [ ]:
!pip install flask_socketio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 


In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

if __name__ == '__main__':
    socketio.run(app,allow_unsafe_werkzeug=True)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [ ]:
@socketio.on('message')
def handle_message(df_attractions_with_similarity_scores):
    print('received message: ' + df_attractions_with_similarity_scores)